In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import taichi as ti
import numpy as np
import math
from engine.mpm_solver import MPMSolver
import argparse
import os




out_dir = 'out_jupyter'
ti.reset()
ti.init(arch=ti.cuda)  # Try to run on GPU

gui = ti.GUI("Taichi Elements", res=512, background_color=0x112F41)

mpm = MPMSolver(res=(256, 256), E_scale=0.001, use_g2p2g=False)
mpm.set_gravity([0, 0])

E1 = 1e6
E2 = 1e3


mpm.add_cube(
    lower_corner=[0.45,0.45],
    cube_size=[0.1,0.1],
    velocity=[0, 0],
    sample_density=0.5,
    material=mpm.material_elastic,
    E=E1,
    nu=0.2
)
for frame in range(10):
    mpm.step(8e-3, print_stat=False)
    particles = mpm.particle_info()
    print(mpm.fan_center, mpm.t, mpm.omega)
    gui.circles(particles['position'], radius=1.5, color=particles['color'])
    gui.show(f'{out_dir}/{frame:06d}.png')

[Taichi] Starting on arch=cuda
[0. 0.] 0.008000000000000018 0.0
[0. 0.] 0.01599999999999917 0.0
[0. 0.] 0.023999999999998325 0.0
[0. 0.] 0.03199999999999748 0.0
[0. 0.] 0.039999999999996635 0.0
[0. 0.] 0.04799999999999579 0.0
[0. 0.] 0.05599999999999494 0.0
[0. 0.] 0.06399999999999544 0.0
[0. 0.] 0.0720000000000017 0.0
[0. 0.] 0.08000000000000797 0.0


: 

In [39]:

ti.init()

field = ti.Vector.field(3,dtype=float,shape=(1,1,1))

# @ti.kernel
def loop():
    for i,j,k in ti.ndrange(1,1,1):
        # field[i,j,k] = i+j+k
        print(field[i,j,k])

loop()

[Taichi] Starting on arch=x64
[0. 0. 0.]


In [3]:
ti.reset()

In [3]:
import taichi as ti
import numpy as np
import engine.utils as utils


# external forces
velocity = 4
frequency = 5
node_x_locs = np.arange(5)
xvalues, fxvalues = utils.rolling_gaussian_generator(velocity, frequency, node_x_locs, resolution=5)


ti.init(arch=ti.cuda)


dim = 2
num_nodes = 9
t_intervals = 5

node_ids_fext = range(5)

f_ext = ti.Vector.field(t_intervals, dtype=ti.f32, shape=(num_nodes,dim+1))


for i, node in enumerate(node_ids_fext):
    f_ext[node,0] = xvalues[i]
    f_ext[node,2] = fxvalues


print(f_ext[2,0])
print(f_ext[2,2])



utils.interpolate(-0.1, f_ext[2,0], f_ext[2,2])

[Taichi] Starting on arch=cuda
[-0.5        -0.5        -0.1         0.30000001  0.69999999]
[0.00000000e+00 2.01636235e-10 3.23329004e-04 4.09432918e-01
 4.09432918e-01]


0.0003233305285110086